<a href="https://colab.research.google.com/github/nunival/flower_classifier/blob/main/Flower_Classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

I've taken some sample code from Tensorflow to use Model Maker with my data to train a flower classification CV model.

In [ ]:
!pip install tflite-model-maker

Import the required packages.

In [ ]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

### Get the data path

I'm directly grabbing the data from the University of Oxford. 

<img src="https://www.robots.ox.ac.uk/~vgg/data/flowers/17/categories.jpg" alt="Upload File" width="800" hspace="100">

In [ ]:
!mkdir "/content/input"
!mkdir "/content/train"

In [ ]:
image_path = tf.keras.utils.get_file(
      '/content/input/flower_photos',
      'https://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz',
      untar=True)

60276736/60270631 [==============================] - 3s 0us/step


In [ ]:
import os, sys, tarfile

def extract(tar_url, extract_path='.'):
    print(tar_url)
    tar = tarfile.open(tar_url, 'r')
    for item in tar:
        tar.extract(item, extract_path)
        if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
            extract(item.name, "./" + item.name[:item.name.rfind('/')])

In [ ]:
extract("/content/input/flower_photos.tar.gz", "/content/input" )

/content/input/flower_photos.tar.gz


The data comes in with no organization. It has jpg with labels of jpg_0001 to jpg_1360. The code below organizes the images into folders with the label name for each folder.

In [ ]:
# organize imports
import os
import glob
import datetime

# print start time
print("[INFO] program started on - " + str(datetime.datetime.now))

# get the input and output path
input_path  = "/content/input/jpg"
output_path = "/content/train"

# get the class label limit
class_limit = 17

# take all the images from the dataset
image_paths = glob.glob(input_path + "/*.jpg")

# variables to keep track
label = 0
i = 0
j = 80

# flower17 class names
class_names = ["daffodil", "snowdrop", "lilyvalley", "bluebell", "crocus",
			   "iris", "tigerlily", "tulip", "fritillary", "sunflower", 
			   "daisy", "coltsfoot", "dandelion", "cowslip", "buttercup",
			   "windflower", "pansy"]

# change the current working directory
os.chdir(output_path)

# loop over the class labels
for x in range(1, class_limit+1):
	# create a folder for that class
  os.system("mkdir " + class_names[label])
	# get the current path
  cur_path = output_path + "/" + class_names[label] + "/"
	# loop over the images in the dataset
  for n in range(i,j):
    os.system("cp " + input_path + "/image_"+ f"{n+1:04d}" +".jpg" + " " + cur_path + "/image_"+ f"{n+1:04d}" +".jpg")

  i += 80
  j += 80
  label += 1

# print end time
print("[INFO] program ended on - " + str(datetime.datetime.now))



[INFO] program started on - <built-in method now of type object at 0x55cb7aa48700>
[INFO] program ended on - <built-in method now of type object at 0x55cb7aa48700>


## Model
Using Model Maker to build my classifier


In [ ]:
data = DataLoader.from_folder("/content/train")
train_data, test_data = data.split(0.8)

INFO:tensorflow:Load image with size: 1360, num_label: 17, labels: bluebell, buttercup, coltsfoot, cowslip, crocus, daffodil, daisy, dandelion, fritillary, iris, lilyvalley, pansy, snowdrop, sunflower, tigerlily, tulip, windflower.


In [ ]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 17)                21777     
Total params: 3,434,801
Trainable params: 21,777
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


34/34 [==============================] - 41s 279ms/step - loss: 2.3285 - accuracy: 0.3640
Epoch 2/5
34/34 [==============================] - 10s 285ms/step - loss: 1.2307 - accuracy: 0.8548
Epoch 3/5
34/34 [==============================] - 10s 283ms/step - loss: 0.9889 - accuracy: 0.9154
Epoch 4/5
34/34 [==============================] - 10s 284ms/step - loss: 0.9133 - accuracy: 0.9320
Epoch 5/5
34/34 [==============================] - 10s 282ms/step - loss: 0.8763 - accuracy: 0.9357


Evaluate the model.

In [ ]:
loss, accuracy = model.evaluate(test_data)

9/9 [==============================] - 4s 302ms/step - loss: 0.9430 - accuracy: 0.9081


Export to TensorFlow Lite model.

Using this model in my app

In [ ]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: /tmp/tmp3thwk81_/assets


INFO:tensorflow:Assets written to: /tmp/tmp3thwk81_/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpj_erpjku/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpj_erpjku/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


In [ ]:
from google.colab import files
files.download('model.tflite') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>